# Generated Model
Let's only consider univariate case. Assume we have $n$ true data points,
$$\{y_i^t,x_i^t\}_{i=1}^n$$
and our underline model is
$$y_i^t = \beta^t x_i^t.$$
Then we could contaminate our data by errors $\delta$ and $\epsilon$,
\begin{align*}
x &= x^t + \delta,\\
y &= y^t + \epsilon.
\end{align*}
Here we assume our error vectors satisfy
* $\text{cov}(\epsilon_i,\epsilon_j)=0$, $\forall i\neq j$, and $\text{Var}(\epsilon_i)=\sigma_\epsilon$.
* $\text{cov}(\delta_i,\delta_j)=0$, $\forall i\neq j$, and $\text{Var}(\delta_i)=\sigma_\delta$.

In [ ]:
using PyPlot;
include("./sampleQ.jl");

In [ ]:
n  = 500;
βᵗ = 2.0;
xᵗ = 10*rand(n);
yᵗ = βᵗ*xᵗ;
ϵ  = 2.0*randn(n);
# δ  = 2.0*randn(n);
τ  = 0.9;
δ  = zeros(n);
for i = 1:n δ[i] = sampleQ(τ); end
x  = xᵗ + δ;
y  = yᵗ + ϵ;

In [ ]:
fid = open("data/data.bin","r")
m,n = read(fid,Int64,2)
x   = read(fid,Float64,m*n)
y   = read(fid,Float64,m)
xᵗ  = read(fid,Float64,m*n)
βᵗ  = read(fid,Float64,n)
τt  = read(fid,Float64,n)[1]
close(fid)

In [ ]:
plot(x,y,".c")
plot(xᵗ,y,"-b")

# Ordinary Least Square
$$\beta_{\text{OLS}} = (x^Tx)^{-1}(x^Ty)$$

In [ ]:
βOLS = dot(x,y)/dot(x,x);

In [ ]:
plot(x,y,".c")
plot(xᵗ,y,"-b");
plot([0,20],[0,βOLS*20],"-g")

# Total Least Square
$$\beta_{\text{TLS}} = \frac{y^Ty-x^Tx+\sqrt{(x^Tx-y^Ty)^2+4(x^Ty)^2}}{2x^Ty}$$

In [ ]:
xx = dot(x,x);
yy = dot(y,y);
xy = dot(x,y);
βTLS = (yy-xx+sqrt((xx-yy)^2.0+4.0*xy^2))/(2.0*xy);

In [ ]:
plot(x,y,".c")
plot(xᵗ,y,"-b");
plot([0,20],[0,βOLS*20],"-g")
plot([0,20],[0,βTLS*20],"-r")

# Total Quantile Regression
There is no closed form solution, since it involves solving a nonlinear equation.
$$\min_{\beta,x_{\text{TQR}}}\rho_\tau(x-x_{\text{TQR}})+\frac{1}{2}\|\beta x_{\text{TQR}}-y\|_2^2$$
The optimality condition is,
$$
\left\{\begin{array}{lll}
\beta^2x_{\text{TQR}}-\beta y\in\partial\rho_\tau(x-x_{\text{TQR}})\\
\beta x_{\text{TQR}}^Tx_{\text{TQR}} = x_{\text{TQR}}^Ty
\end{array}\right.
$$
But notice that if we fix $\beta$ this is just the Moreau envelope. Therefore we could easily project out $x_{\text{TQR}}$ and do the iteration on $\beta$ to obtain the optimal solution.

In [ ]:
function projx!(xTQR, βTQR, x, y, τ)
    n = length(x);
    for i = 1:n
        a = (βTQR*y[i]-τ)/βTQR^2.;
        b = a + 1.0/βTQR^2.;
        xTQR[i] = min(max(x[i],a),b);
    end
end
function objTQR(xTQR, βTQR, x, y, τ)
    n = length(x);
    val = 0.0;
    for i = 1:n
        val += quantPen(x[i]-xTQR[i],τ)
    end
    val += 0.5*vecnorm(βTQR*xTQR-y)^2
    return val
end

In [ ]:
xTQR = zeros(m);
βTQR = 1.0;
τTQR = 0.9;
for k = 1:2000
    projx!(xTQR, βTQR, x, y, τTQR);
    βTQR = dot(y,xTQR)/dot(xTQR,xTQR);
end
@show(βTQR);
@show(objTQR(xTQR, βTQR, x, y, τTQR));

In [ ]:
plot(x,y,".c")
plot(xᵗ,y,"-b");
plot([0,20],[0,βOLS*20],"-g")
plot([0,20],[0,βTLS*20],"-r")
plot([0,1],[0,βTQR*1],"-m")

## Adding $\tau$ to be unknow

In [ ]:
function projτ(xTQR, x)
    a = (sum(x) - sum(xTQR))/length(x);
    a == 0.0 ? (return 0.5) : (return 2.0/(sqrt(a^2.0+4.0)-a+2.0))
end
function objTQRT(xTQR, βTQR, x, y, τTQR)
    n = length(x);
    val = 0.0;
    for i = 1:n
        val += quantPen(x[i]-xTQR[i],τTQR)
    end
    val += 0.5*vecnorm(βTQR*xTQR-y)^2
    val -= n*(log(τTQR*(1-τTQR)))
    return val
end

In [ ]:
xTQR = copy(x);
βTQR = 1.0;
τTQR = 0.5;
for k = 1:500
    τTQR = projτ(xTQR,x);
#     @printf("after τ, obj %1.5e, τ = %1.5e\n",objTQRT(xTQR,βTQR,x,y,τTQR),τTQR)
    projx!(xTQR, βTQR, x, y, τTQR);
#     @printf("after x, obj %1.5e\n",objTQRT(xTQR,βTQR,x,y,τTQR))
    βTQR = dot(y,xTQR)/dot(xTQR,xTQR);
#     @printf("after β, obj %1.5e, β = %1.5e\n",objTQRT(xTQR,βTQR,x,y,τTQR),βTQR)
    k%20==0 &&
    @printf("iter %3d, obj %1.5e\n",k,objTQRT(xTQR,βTQR,x,y,τTQR))
end
@show(τTQR);
@show(βTQR);

In [ ]:
βᵗ, βOLS, βTLS, βTQR

In [ ]:
for β ∈ [βᵗ[1], βOLS, βTLS, βTQR]
    @printf("relative error: %1.5e\n",abs(β-βᵗ[1])/abs(βᵗ[1]))
end